# AMD - Network models
## Exercise 2

Rent-a-car company with 4 destinations $\{A, B, C, D\}$ and 3 supplies $\{1, 2, 3\}$ with equal values of demand and offer.

In [1]:
using JuMP
using Cbc

In [19]:
m = Model(Cbc.Optimizer) 
@variable(m, x[1:3, 1:4] >= 0)
#balance restritions
demand = [2 3 5 7]
@constraint(m, sum(x[1:3, i=1:4]) == demand[i])

offer = [6 1 10]
for i in 1:3
    @constraint(m, sum(x[i, 1:4]) == offer[i])
end

#costs
C = [ 7 11 3 2; 
1 6 0 1;
9 15 8 5]
@objective(m, Min, sum(x .* C))
println(m)
optimize!(m)

Min 7 x[1,1] + x[2,1] + 9 x[3,1] + 11 x[1,2] + 6 x[2,2] + 15 x[3,2] + 3 x[1,3] + 8 x[3,3] + 2 x[1,4] + x[2,4] + 5 x[3,4]
Subject to
 x[1,1] + x[2,1] + x[3,1] == 2.0
 x[1,2] + x[2,2] + x[3,2] == 3.0
 x[1,3] + x[2,3] + x[3,3] == 5.0
 x[1,4] + x[2,4] + x[3,4] == 7.0
 x[1,1] + x[1,2] + x[1,3] + x[1,4] == 6.0
 x[2,1] + x[2,2] + x[2,3] + x[2,4] == 1.0
 x[3,1] + x[3,2] + x[3,3] + x[3,4] == 10.0
 x[1,1] >= 0.0
 x[2,1] >= 0.0
 x[3,1] >= 0.0
 x[1,2] >= 0.0
 x[2,2] >= 0.0
 x[3,2] >= 0.0
 x[1,3] >= 0.0
 x[2,3] >= 0.0
 x[3,3] >= 0.0
 x[1,4] >= 0.0
 x[2,4] >= 0.0
 x[3,4] >= 0.0

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 7 (0) rows, 12 (0) columns and 24 (0) elements
0  Obj 0 Primal inf 33.999999 (7)
9  Obj 100
Optimal - objective value 100
Optimal objective 100 - 9 iterations time 0.002
Total time (CPU seconds):       0.11   (Wallclock seconds):       0.11



In [26]:
m = Model(Cbc.Optimizer) 
@variable(m, x[1:3, 1:4] >= 0)
#balance restritions
demand = [2 3 5 7]
@constraint(m, con[i = 1:4], sum(x[1:3, i]) == demand[i])

offer = [6 1 10]
@constraint(m, con_offer[i = 1:3], sum(x[i, 1:4]) == offer[i])

#costs
C = [ 7 11 3 2; 
1 6 0 1;
9 15 8 5]
@objective(m, Min, sum(x .* C))
println(m)
optimize!(m)

Min 7 x[1,1] + x[2,1] + 9 x[3,1] + 11 x[1,2] + 6 x[2,2] + 15 x[3,2] + 3 x[1,3] + 8 x[3,3] + 2 x[1,4] + x[2,4] + 5 x[3,4]
Subject to
 con[1] : x[1,1] + x[2,1] + x[3,1] == 2.0
 con[2] : x[1,2] + x[2,2] + x[3,2] == 3.0
 con[3] : x[1,3] + x[2,3] + x[3,3] == 5.0
 con[4] : x[1,4] + x[2,4] + x[3,4] == 7.0
 con_offer[1] : x[1,1] + x[1,2] + x[1,3] + x[1,4] == 6.0
 con_offer[2] : x[2,1] + x[2,2] + x[2,3] + x[2,4] == 1.0
 con_offer[3] : x[3,1] + x[3,2] + x[3,3] + x[3,4] == 10.0
 x[1,1] >= 0.0
 x[2,1] >= 0.0
 x[3,1] >= 0.0
 x[1,2] >= 0.0
 x[2,2] >= 0.0
 x[3,2] >= 0.0
 x[1,3] >= 0.0
 x[2,3] >= 0.0
 x[3,3] >= 0.0
 x[1,4] >= 0.0
 x[2,4] >= 0.0
 x[3,4] >= 0.0

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 7 (0) rows, 12 (0) columns and 24 (0) elements
0  Obj 0 Primal inf 33.999999 (7)
9  Obj 100
Optimal - objective value 100
Optimal objective 100 - 9 iterations time 0.002
Total time (CPU seconds):   

In [27]:
println("Result")
JuMP.value.(x)

Result


3×4 Array{Float64,2}:
 0.0  1.0  5.0  0.0
 0.0  1.0  0.0  0.0
 2.0  1.0  0.0  7.0

## Exercise 3
NASDAQ - transaction of informations about operations.

In [31]:
function matrix_method(cost, demand, supply)
    #In the cost matrix, each row represents a supply, each column represents a demand
    #The quantitys for each vertex to be completed
    demand_remaining = [d for d in demand]
    supply_remaining = [s for s in supply]
    #Indicating if some vertex if already fullfield
    demand_bool = [false for _ in demand]
    supply_bool = [false for _ in supply]
    #Initial variable
    x0 = [0 for _ in cost]
    value, edge = findmin(cost)
    
    while sum(demand_remaining) + sum(supply_remaining) > 0
        cur_supply = getindex(edge, 1)
        cur_demand = getindex(edge, 2)
        #Can only move the minimun between the supply and demand 
        edge_value = minimum([demand_remaining[cur_demand], supply_remaining[cur_supply]])
        
        #Update vectors
        demand_remaining[cur_demand] -= edge_value
        supply_remaining[cur_supply] -= edge_value
        x0[cur_supply, cur_demand] = edge_value
        
        #Check if the updated vertex is complete
        if demand_remaining[cur_demand]  == 0
            demand_bool[cur_demand] = true
        end
        if supply_remaining[cur_supply] == 0
            supply_bool[cur_supply] = true
        end
        
        #Little trick to don't find arcs of completed vertex
        cost[supply_bool, :] .= Inf
        cost[:, demand_bool] .= Inf
        value, edge = findmin(cost)
    end
    return x0
end

matrix_method (generic function with 1 method)

In [35]:
costs = float([6 6 9 4 10;
3 2 7 5 12;
8 7 5 6 4;
11 12 9 5 2;
4 3 4 5 11])
demand = [120, 80, 50, 75, 85]
supply = [45, 90, 95, 75, 105];

x0 = matrix_method(float(costs), demand, supply)

5×5 Array{Int64,2}:
   0   0   0  45   0
  10  80   0   0   0
   5   0  50  30  10
   0   0   0   0  75
 105   0   0   0   0

In [53]:
costs = float([6 6 9 4 10;
3 2 7 5 12;
8 7 5 6 4;
11 12 9 5 2;
4 3 4 5 11])
demand = [120, 80, 50, 75, 85]
supply = [45, 90, 95, 75, 105];

In [54]:
# Supply
# [Clev, Bost, Houst, Los A, Wash]
# Demand
# [Trum, NY, Atla, Chi, SanF]

m = Model(Cbc.Optimizer)

@variable(m, x[1:5, 1:5] >= 0)
set_start_value.(x, x0)

@constraint(m, const_demand[i=1:5], sum(x[:, i]) == demand[i])

@constraint(m, const_supply[i=1:5], sum(x[i, :]) == supply[i])

@objective(m, Min, sum(x .* costs))
println(m)
optimize!(m)

Min 6 x[1,1] + 3 x[2,1] + 8 x[3,1] + 11 x[4,1] + 4 x[5,1] + 6 x[1,2] + 2 x[2,2] + 7 x[3,2] + 12 x[4,2] + 3 x[5,2] + 9 x[1,3] + 7 x[2,3] + 5 x[3,3] + 9 x[4,3] + 4 x[5,3] + 4 x[1,4] + 5 x[2,4] + 6 x[3,4] + 5 x[4,4] + 5 x[5,4] + 10 x[1,5] + 12 x[2,5] + 4 x[3,5] + 2 x[4,5] + 11 x[5,5]
Subject to
 const_demand[1] : x[1,1] + x[2,1] + x[3,1] + x[4,1] + x[5,1] == 120.0
 const_demand[2] : x[1,2] + x[2,2] + x[3,2] + x[4,2] + x[5,2] == 80.0
 const_demand[3] : x[1,3] + x[2,3] + x[3,3] + x[4,3] + x[5,3] == 50.0
 const_demand[4] : x[1,4] + x[2,4] + x[3,4] + x[4,4] + x[5,4] == 75.0
 const_demand[5] : x[1,5] + x[2,5] + x[3,5] + x[4,5] + x[5,5] == 85.0
 const_supply[1] : x[1,1] + x[1,2] + x[1,3] + x[1,4] + x[1,5] == 45.0
 const_supply[2] : x[2,1] + x[2,2] + x[2,3] + x[2,4] + x[2,5] == 90.0
 const_supply[3] : x[3,1] + x[3,2] + x[3,3] + x[3,4] + x[3,5] == 95.0
 const_supply[4] : x[4,1] + x[4,2] + x[4,3] + x[4,4] + x[4,5] == 75.0
 const_supply[5] : x[5,1] + x[5,2] + x[5,3] + x[5,4] + x[5,5] == 105.0
 x[1,

In [55]:
display("Original solution with minimization matrix method")
display(x0)
display("Found solution")
display(JuMP.value.(x))

"Original solution with minimization matrix method"

5×5 Array{Int64,2}:
   0   0   0  45   0
  10  80   0   0   0
   5   0  50  30  10
   0   0   0   0  75
 105   0   0   0   0

"Found solution"

5×5 Array{Float64,2}:
   0.0   0.0   0.0  45.0   0.0
  15.0  75.0   0.0   0.0   0.0
   0.0   5.0  50.0  30.0  10.0
   0.0   0.0   0.0   0.0  75.0
 105.0   0.0   0.0   0.0   0.0

In [49]:
function matrix_bad_method(cost, demand, supply)
    #In the cost matrix, each row represents a supply, each column represents a demand
    #The quantitys for each vertex to be completed
    demand_remaining = [d for d in demand]
    supply_remaining = [s for s in supply]
    #Indicating if some vertex if already fullfield
    demand_bool = [false for _ in demand]
    supply_bool = [false for _ in supply]
    #Initial variable
    x0 = [0 for _ in cost]
    value, edge = findmax(cost)
    
    while sum(demand_remaining) + sum(supply_remaining) > 0
        cur_supply = getindex(edge, 1)
        cur_demand = getindex(edge, 2)
        #Can only move the minimun between the supply and demand 
        edge_value = minimum([demand_remaining[cur_demand], supply_remaining[cur_supply]])
        
        #Update vectors
        demand_remaining[cur_demand] -= edge_value
        supply_remaining[cur_supply] -= edge_value
        x0[cur_supply, cur_demand] = edge_value
        
        #Check if the updated vertex is complete
        if demand_remaining[cur_demand]  == 0
            demand_bool[cur_demand] = true
        end
        if supply_remaining[cur_supply] == 0
            supply_bool[cur_supply] = true
        end
        
        #Little trick to don't find arcs of completed vertex
        cost[supply_bool, :] .= 0
        cost[:, demand_bool] .= 0
        value, edge = findmax(cost)
    end
    return x0
end

matrix_bad_method (generic function with 1 method)

In [50]:
costs = float([6 6 9 4 10;
3 2 7 5 12;
8 7 5 6 4;
11 12 9 5 2;
4 3 4 5 11])
demand = [120, 80, 50, 75, 85]
supply = [45, 90, 95, 75, 105];
x0_bad = matrix_bad_method(float(costs), demand, supply)

5×5 Array{Int64,2}:
  0   0  45   0   0
  0   0   5   0  85
 95   0   0   0   0
  0  75   0   0   0
 25   5   0  75   0

In [51]:
costs = float([6 6 9 4 10;
3 2 7 5 12;
8 7 5 6 4;
11 12 9 5 2;
4 3 4 5 11])
demand = [120, 80, 50, 75, 85]
supply = [45, 90, 95, 75, 105];

In [52]:
# Supply
# [Clev, Bost, Houst, Los A, Wash]
# Demand
# [Trum, NY, Atla, Chi, SanF]

m = Model(Cbc.Optimizer)

@variable(m, x[1:5, 1:5] >= 0)
set_start_value.(x, x0_bad)

demand = [120 80 50 75 85]
@constraint(m, const_demand[i=1:5], sum(x[:, i]) == demand[i])

supply = [45 90 95 75 105]
@constraint(m, const_supply[i=1:5], sum(x[i, :]) == supply[i])

costs = [6 6 9 4 10;
3 2 7 5 12;
8 7 5 6 4;
11 12 9 5 2;
4 3 4 5 11]
@objective(m, Min, sum(x .* costs))
println(m)
optimize!(m)

Min 6 x[1,1] + 3 x[2,1] + 8 x[3,1] + 11 x[4,1] + 4 x[5,1] + 6 x[1,2] + 2 x[2,2] + 7 x[3,2] + 12 x[4,2] + 3 x[5,2] + 9 x[1,3] + 7 x[2,3] + 5 x[3,3] + 9 x[4,3] + 4 x[5,3] + 4 x[1,4] + 5 x[2,4] + 6 x[3,4] + 5 x[4,4] + 5 x[5,4] + 10 x[1,5] + 12 x[2,5] + 4 x[3,5] + 2 x[4,5] + 11 x[5,5]
Subject to
 const_demand[1] : x[1,1] + x[2,1] + x[3,1] + x[4,1] + x[5,1] == 120.0
 const_demand[2] : x[1,2] + x[2,2] + x[3,2] + x[4,2] + x[5,2] == 80.0
 const_demand[3] : x[1,3] + x[2,3] + x[3,3] + x[4,3] + x[5,3] == 50.0
 const_demand[4] : x[1,4] + x[2,4] + x[3,4] + x[4,4] + x[5,4] == 75.0
 const_demand[5] : x[1,5] + x[2,5] + x[3,5] + x[4,5] + x[5,5] == 85.0
 const_supply[1] : x[1,1] + x[1,2] + x[1,3] + x[1,4] + x[1,5] == 45.0
 const_supply[2] : x[2,1] + x[2,2] + x[2,3] + x[2,4] + x[2,5] == 90.0
 const_supply[3] : x[3,1] + x[3,2] + x[3,3] + x[3,4] + x[3,5] == 95.0
 const_supply[4] : x[4,1] + x[4,2] + x[4,3] + x[4,4] + x[4,5] == 75.0
 const_supply[5] : x[5,1] + x[5,2] + x[5,3] + x[5,4] + x[5,5] == 105.0
 x[1,

In [56]:
display("Original solution with maximization matrix method")
display(x0_bad)
display("Found solution")
display(JuMP.value.(x))


"Original solution with maximization matrix method"

5×5 Array{Int64,2}:
  0   0  45   0   0
  0   0   5   0  85
 95   0   0   0   0
  0  75   0   0   0
 25   5   0  75   0

"Found solution"

5×5 Array{Float64,2}:
   0.0   0.0   0.0  45.0   0.0
  15.0  75.0   0.0   0.0   0.0
   0.0   5.0  50.0  30.0  10.0
   0.0   0.0   0.0   0.0  75.0
 105.0   0.0   0.0   0.0   0.0

Without the good initial value it takes the same amount of iterations to get to the result.

## Exercise 13

Production of calculators for six months $\{jan, \dots, jun\}$ with a regular production and a over-time production.

In [10]:
m = Model(Cbc.Optimizer)

#regular supply
@variable(m, regular[1:6] >= 0)

#over-time supply
@variable(m, overtime[1:6] >= 0)

#month-to-month supply
@variable(m, month_month[1:5] >= 0)

#normal production constraint
@constraint(m, regular_const[i = 1:6], regular[i] <= 100)

#over-time production constraint
@constraint(m, overtime_const[i = 1:6], overtime[i] <= 75)

demand = [100 150 200 100 200 150]
#balance constraint
@constraint(m, demand_jan, regular[1] + overtime[1] - month_month[1] == demand[1]) 
@constraint(m, demand_const[i = 2:5], regular[i] + overtime[i] - month_month[i] + month_month[i - 1] >= demand[i])
@constraint(m, demand_jun, regular[6] + overtime[6] + month_month[5] == demand[6])

@objective(m, Min, sum(regular * 10) + sum(overtime * 12) + sum(month_month * 0.5))
println(m)
optimize!(m)

Min 10 regular[1] + 10 regular[2] + 10 regular[3] + 10 regular[4] + 10 regular[5] + 10 regular[6] + 12 overtime[1] + 12 overtime[2] + 12 overtime[3] + 12 overtime[4] + 12 overtime[5] + 12 overtime[6] + 0.5 month_month[1] + 0.5 month_month[2] + 0.5 month_month[3] + 0.5 month_month[4] + 0.5 month_month[5]
Subject to
 demand_jan : regular[1] + overtime[1] - month_month[1] == 100.0
 demand_jun : regular[6] + overtime[6] + month_month[5] == 150.0
 demand_const[2] : regular[2] + overtime[2] + month_month[1] - month_month[2] >= 150.0
 demand_const[3] : regular[3] + overtime[3] + month_month[2] - month_month[3] >= 200.0
 demand_const[4] : regular[4] + overtime[4] + month_month[3] - month_month[4] >= 100.0
 demand_const[5] : regular[5] + overtime[5] + month_month[4] - month_month[5] >= 200.0
 regular_const[1] : regular[1] <= 100.0
 regular_const[2] : regular[2] <= 100.0
 regular_const[3] : regular[3] <= 100.0
 regular_const[4] : regular[4] <= 100.0
 regular_const[5] : regular[5] <= 100.0
 regul

In [21]:
display("Regular production")
display(JuMP.value.(regular))
display("Overtime production")
display(JuMP.value.(overtime))
display("Month to month passage")
display(JuMP.value.(month_month))
display("Total month production")
display(JuMP.value.(regular) + JuMP.value.(overtime) + [0; JuMP.value.(month_month)])

"Produção regular"

6-element Array{Float64,1}:
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0

"Produção overtime"

6-element Array{Float64,1}:
  0.0
 75.0
 75.0
 25.0
 75.0
 50.0

"Passagem mês para mês"

5-element Array{Float64,1}:
  0.0
 25.0
  0.0
 25.0
  0.0

"Produção total em cada mês"

6-element Array{Float64,1}:
 100.0
 175.0
 200.0
 125.0
 200.0
 150.0

## Exercise 19

In [4]:
S = [5, 5, 6]
D = [2, 2, 4, 4, 3]

m = Model(Cbc.Optimizer)

@variable(m, x[1:3, 1:5] >= 0)

@constraint(m, const_demand[i=1:5], sum(x[:, i]) == D[i])

@constraint(m, const_supply[i=1:3], sum(x[i, :]) <= S[i])

cost = [2 1 2 3 3;
2 2 2 1 -1;
3 3 2 1 2]
@objective(m, Min, sum(x .* cost))
println(m)
optimize!(m)

Min 2 x[1,1] + 2 x[2,1] + 3 x[3,1] + x[1,2] + 2 x[2,2] + 3 x[3,2] + 2 x[1,3] + 2 x[2,3] + 2 x[3,3] + 3 x[1,4] + x[2,4] + x[3,4] + 3 x[1,5] - x[2,5] + 2 x[3,5]
Subject to
 const_demand[1] : x[1,1] + x[2,1] + x[3,1] == 2.0
 const_demand[2] : x[1,2] + x[2,2] + x[3,2] == 2.0
 const_demand[3] : x[1,3] + x[2,3] + x[3,3] == 4.0
 const_demand[4] : x[1,4] + x[2,4] + x[3,4] == 4.0
 const_demand[5] : x[1,5] + x[2,5] + x[3,5] == 3.0
 const_supply[1] : x[1,1] + x[1,2] + x[1,3] + x[1,4] + x[1,5] <= 5.0
 const_supply[2] : x[2,1] + x[2,2] + x[2,3] + x[2,4] + x[2,5] <= 5.0
 const_supply[3] : x[3,1] + x[3,2] + x[3,3] + x[3,4] + x[3,5] <= 6.0
 x[1,1] >= 0.0
 x[2,1] >= 0.0
 x[3,1] >= 0.0
 x[1,2] >= 0.0
 x[2,2] >= 0.0
 x[3,2] >= 0.0
 x[1,3] >= 0.0
 x[2,3] >= 0.0
 x[3,3] >= 0.0
 x[1,4] >= 0.0
 x[2,4] >= 0.0
 x[3,4] >= 0.0
 x[1,5] >= 0.0
 x[2,5] >= 0.0
 x[3,5] >= 0.0

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Pr

In [6]:
@show JuMP.value.(x)

JuMP.value.(x) = [2.0 2.0 1.0 0.0 0.0; 0.0 0.0 0.0 2.0 3.0; 0.0 0.0 3.0 2.0 0.0]


3×5 Array{Float64,2}:
 2.0  2.0  1.0  0.0  0.0
 0.0  0.0  0.0  2.0  3.0
 0.0  0.0  3.0  2.0  0.0